In [1]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [2]:
# Torch loads
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data_utils
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

# Non-Torch Loads
import pandas as pd
import numpy as np

#Cleaning Loads
import regex as re
import emoji
import nltk

In [3]:
train = pd.read_csv("nlp-getting-started/train.csv")
train = train.drop(labels = ["keyword", "location", "id"], axis = 1)
print(train.head())
(train.shape)

                                                text  target
0  Our Deeds are the Reason of this #earthquake M...       1
1             Forest fire near La Ronge Sask. Canada       1
2  All residents asked to 'shelter in place' are ...       1
3  13,000 people receive #wildfires evacuation or...       1
4  Just got sent this photo from Ruby #Alaska as ...       1


(7613, 2)

# EDA
The dataset has 57% non-disaster tweets, and 43% Disaster tweets.  There are 31924 unique words.  This will drive my tuning the vectorization of the model.

In [4]:
train['target'].describe()

count    7613.00000
mean        0.42966
std         0.49506
min         0.00000
25%         0.00000
50%         0.00000
75%         1.00000
max         1.00000
Name: target, dtype: float64

In [5]:
unique_words = set()
for text in train["text"]:
    unique_words.update(text.split())
print(len(unique_words))

31924


# Cleaning
Standard tweet cleaning.  Cleaning found at:
https://stackoverflow.com/questions/64719706/cleaning-twitter-data-pandas-python

# Tokenizatin
I tolkenized the tweets in preparation to convert to tensors for embedding.


In [6]:
for tweet in train['text']:
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = emoji.replace_emoji(tweet, '') #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    #Tolkenize the Text
    
    tknzr = nltk.tokenize.casual.TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
    tweet = tknzr.tokenize(tweet)

In [7]:
# Split the data into train and test sets
train_dat = train.sample(frac = .8)
test_dat = train.drop(train_dat.index)

# Vectorizing
I used pytorchs internal vectorizer to vectorize the text.  I


In [8]:
# Dataset Importing from https://pytorch.org/tutorials/beginner/basics/data_tutorial.html?highlight=dataloader

#tokenize the text for tensor load
def tokenize(text):
    # Your tokenizer logic here
    return [vocab[token] for token in text.split()]

class CustomTextDataset(Dataset):
    def __init__(self, data,vocab, transform=None, target_transform=None):
        self.data = data
        self.vocab = vocab
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tweet = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 1]
        tokenized_tweet = [self.vocab.get(token, 0) for token in tweet.split()]  # Tokenize and handle unknown tokens
        tweet_tensor = torch.tensor(tokenized_tweet, dtype=torch.long)
        label_tensor = torch.tensor(label, dtype=torch.float)
        
        if self.transform:
            tweet_tensor = self.transform(tweet_tensor)
        if self.target_transform:
            label_tensor = self.target_transform(label_tensor)

        return tweet_tensor, label_tensor

def collate_batch(batch):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        text_list.append(_text.clone().detach())
    return pad_sequence(text_list, batch_first=True), torch.tensor(label_list, dtype=torch.float)

vocab = {'Emergency': 1, 'Non-Emergency':2}

# Creating training and validation datasets
training_set = CustomTextDataset(train_dat, vocab)
validation_set = CustomTextDataset(test_dat, vocab)

# Creating data loaders
training_loader = torch.utils.data.DataLoader(training_set, batch_size=32, shuffle=True, collate_fn=collate_batch)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=32, shuffle=False, collate_fn=collate_batch)


In [38]:
# Initial Setup

class Tweets(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Tweets, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn_layer = nn.RNN(input_size=embedding_dim, hidden_size=32, num_layers=1,
                                nonlinearity='tanh', bias=True, bidirectional=False)
        self.output_layer = nn.Linear(32, 1)

    def forward(self, input_text):
        # Assuming input_text is already a LongTensor that has been prepared outside the model
        embedded = self.embedding(input_text)
        rnn_output, _ = self.rnn_layer(embedded)
        output = rnn_output[:, -1, :]  # Extract the last timestep output
        final_output = self.output_layer(output)
        return final_output.squeeze()  # Squeeze to remove the extra dimension


In [ ]:


def train_one_epoch(epoch_index, model,training_loader, loss_fn, optimizer, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

# Execution
model = Tweets(vocab_size=30000, embedding_dim =1024)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
tb_writer = SummaryWriter('logs')
train_one_epoch(1,model, training_loader, loss_fn, optimizer, tb_writer)
tb_writer.close()